In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
138,tt2278388,30000000,174600318,The Grand Budapest Hotel,Ralph Fiennes|Tony Revolori|F. Murray Abraham|...,Wes Anderson,A perfect holiday without leaving home.,The Grand Budapest Hotel tells of a legendary ...,99,Comedy|Drama,Fox Searchlight Pictures|Scott Rudin Productio...,2/26/2014,7.9,2014
1247,tt2024432,35000000,173965010,Identity Thief,Jason Bateman|Melissa McCarthy|Jon Favreau|Ama...,Seth Gordon,She's having the time of his life,When a mild-mannered businessman learns his id...,111,Comedy|Crime,Stuber Productions|Aggregate Films|DumbDumb,2/7/2013,5.6,2013
1452,tt0327084,80000000,155019340,Over the Hedge,Bruce Willis|Garry Shandling|Steve Carell|Wand...,Tim Johnson|Karey Kirkpatrick,Taking back the neighborhood... One snack at a...,A scheming raccoon fools a mismatched family o...,83,Comedy|Animation|Family,Pacific Data Images (PDI)|DreamWorks Animation,4/22/2006,6.2,2006
1275,tt0790628,30000000,27437881,The Incredible Burt Wonderstone,Steve Carell|Jim Carrey|Steve Buscemi|Olivia W...,Don Scardino,Abracatastic!,After breaking up with his longtime stage part...,100,Comedy,New Line Cinema|Benderspink|Carousel Productions,3/14/2013,5.2,2013
236,tt2319580,12700000,3430018,The Grand Seduction,Brendan Gleeson|Taylor Kitsch|Gordon Pinsent|L...,Don McKellar,The small town of Tickle Cove needed a doctor....,A small fishing village must procure a local d...,112,Comedy,Max Films Productions|Morag Loves Company,5/30/2014,6.7,2014


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов
data1 = data.copy()  # создадим копию датафрэйма для избежания перемешивания данных в строчках

# Добавляем столбцы:

data['profit'] = data['revenue'] - data['budget']  # столбец с прибылью
data['month'] = pd.DatetimeIndex(data['release_date']).month  # столбец с месяцами релиза
data['title_length'] = data['original_title'].map(lambda x: len(x))  # столбец с длиной названия
data['overview_length'] = data['overview'].map(lambda x: len(x))  # столбец с длинной описания

# Преобразуем столбцы:

data.release_date = pd.to_datetime(data.release_date)  # приведем данные в столбце release_date к типу datetime

# 1. У какого фильма из списка самый большой бюджет?

In [5]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'  # +

In [6]:
data[data.budget == data.budget.max()].original_title  # используем маску

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

#### ВАРИАНТ 2

In [7]:
data.sort_values(by='budget', ascending=False).head(1).original_title  # используем sort_values

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
answers['2'] = 'Gods and Generals (tt0279111)'  # +

In [9]:
data[data.runtime == data.runtime.max()].original_title  # используем маску

1157    Gods and Generals
Name: original_title, dtype: object

#### ВАРИАНТ 2

In [10]:
data.sort_values(by='runtime', ascending=False).head(1).original_title  # используем sort_values

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [11]:
answers['3'] = 'Winnie the Pooh (tt1449283)'  # +

In [12]:
data[data.runtime == data.runtime.min()].original_title  # используем маску

768    Winnie the Pooh
Name: original_title, dtype: object

#### ВАРИАНТ 2

In [13]:
data.sort_values(by='runtime').head(1).original_title  # используем sort_values

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [14]:
answers['4'] = '110'  # +

In [15]:
round(data.runtime.mean())  # округляем для соответствия ответу

110

# 5. Каково медианное значение длительности фильмов? 

In [16]:
answers['5'] = '107'  # +

In [17]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [18]:
answers['6'] = 'Avatar (tt0499549)'  # +

In [19]:
# код получения столбца profit вынеcен в Предобработку

In [20]:
data[data.profit == data.profit.max()].original_title

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [21]:
 answers['7'] = 'The Lone Ranger (tt1210819)'  # +

In [22]:
data[data.profit == data.profit.min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
 answers['8'] = '1478'  # +

In [24]:
len(data[data.revenue > data.budget])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [25]:
 answers['9'] = 'The Dark Knight (tt0468569)'  # +

In [26]:
data[data.release_year == 2008].query('revenue == revenue.max()').original_title

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [27]:
 answers['10'] = 'The Lone Ranger (tt1210819)'  # +

In [28]:
data[(data.release_year > 2011) & (data.release_year < 2015)].query('profit == profit.min()').original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [29]:
answers['11'] = 'Drama'  # +

In [30]:
#dataframe data1 - копия data, сделанная в предобработке

In [31]:
data1.genres = data1.genres.str.split('|')  # убираем разделитель в столбце
c = Counter()  # словарь для жанров
# считаем фильмы в каждом жанре и пишем их в словарь
for i in range(0, len(data)):  
    for genres in data1.genres.iloc[i]:
        c[genres] += 1
c

Counter({'Action': 582,
         'Adventure': 415,
         'Science Fiction': 248,
         'Thriller': 596,
         'Fantasy': 222,
         'Crime': 315,
         'Western': 19,
         'Drama': 782,
         'Family': 260,
         'Animation': 139,
         'Comedy': 683,
         'Mystery': 168,
         'Romance': 308,
         'War': 58,
         'History': 62,
         'Music': 64,
         'Horror': 176,
         'Documentary': 8,
         'Foreign': 2})

In [32]:
max(c, key=c.get)  # ищем жанр с максимальным значением

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
answers['12'] = 'Drama'  # +

In [34]:
genres_list = []  # список для жанров
# ищем прибыльные жанры и пишем их в список
for genre in data[data.profit > 0].genres:
    genres_list.append(genre.split('|'))
genres_list = sum(genres_list, [])
pd.DataFrame(genres_list).value_counts().head(1)

Drama    560
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [35]:
answers['13'] = 'Peter Jackson'  # +

In [36]:
directors = set(data1.director.str.split('|').sum()) # собираем режиссеров в множество
# пишем в серию режиссера и кассовые сборы, сортируем по убыванию
pd.Series({x: data1[data1.director.str.contains(x)].revenue.sum() for x in directors}).sort_values(ascending = False).head(1)

Peter Jackson    6490593685
dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [37]:
answers['14'] = 'Robert Rodriguez'  # +

In [38]:
data1 = data.copy()  # здесь и далее обновил dataframe, т.к. он уже изменен выше
data1[data1.genres.str.contains('Action')].director.str.split('|').explode('director').mode()[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [39]:
answers['15'] = 'Chris Hemsworth'  # +

In [40]:
cols = ['cast', 'revenue', 'original_title']  # сбираем нужные нам столбцы
dt = data.query('release_year == 2012')[cols]  # берем нужный нам год
# Группируем по актёрам и по кассовым сборам фильмов с их участием:
dt['cast'] = dt.cast.str.split('|')  # убираем разделитель
dt = dt.explode('cast')  # разбиваем всех актеров по одному
dt = dt.groupby(['cast'])[['revenue']].sum().sort_values(by = 'revenue', ascending=False)
dt.head(1)

,revenue
cast,
Chris Hemsworth,2027450773


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [41]:
answers['16'] = 'Matt Damon'  # +

In [42]:
data[data.budget>data.budget.mean()].cast.str.split('|').explode().mode()[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [43]:
answers['17'] = 'Action'  # +

In [44]:
data[data.cast.str.contains('Nicolas Cage')].genres.str.split('|').explode().mode()[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [45]:
answers['18'] = 'K-19: The Widowmaker'  # +

In [46]:
data[data.production_companies.str.contains('Paramount Pictures')].sort_values(by='profit').head(1).original_title

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [47]:
answers['19'] = '2015'  # +

In [48]:
data[['release_year', 'revenue']].groupby(by='release_year').sum().sort_values(by='revenue', ascending=False).head(1)

,revenue
release_year,
2015,25449202382


# 20. Какой самый прибыльный год для студии Warner Bros?

In [49]:
answers['20'] = '2014'  # +

In [50]:
data[data.production_companies.str.contains('Warner Bros')
].groupby(by='release_year').sum().sort_values(by='profit', ascending=False)['profit'].head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [51]:
answers['21'] = 'сентябрь'  # +

In [52]:
# столбец month добавлен в предобработке

In [53]:
data_sum = data.month.value_counts()
data_sum.head(1)

9    227
Name: month, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [54]:
answers['22'] = '450'  # +

In [55]:
data[data.release_date.dt.month.isin([6, 7, 8,])].shape

(450, 18)

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [56]:
answers['23'] = 'Peter Jackson'  # +

In [57]:
data[data.release_date.dt.month.isin([1, 2,12])].director.str.split('|').explode('director').value_counts().head(1)

Peter Jackson    7
Name: director, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [58]:
answers['24'] = 'Four By Two Productions'  # +

In [59]:
prod_comp = data['production_companies'].str.split('|').explode().unique()  # список уникальных названий компаний
sum_gen = pd.Series(dtype='float64', index=prod_comp)  # серия для хранения средних значений
# вычислим среднее значение для всех названий фильмов созданных компанией
for comp in prod_comp:
    sum_gen[comp] = data['title_length'][data['production_companies'].map(lambda x: True if comp in x else False)].mean()
sum_gen.sort_values(ascending=False).head(1)

Four By Two Productions    83.0
dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [60]:
answers['25'] = 'Midnight Picture Show'  # +

In [61]:
prod_comp = data['production_companies'].str.split('|').explode().unique()  # список уникальных названий компаний
sum_gen = pd.Series(dtype='float64', index=prod_comp)  # серия для хранения средних значений
# вычислим среднее значение для всех описаний фильмов созданных компанией
for comp in prod_comp:
    sum_gen[comp] = data['overview_length'][data['production_companies'].map(lambda x: True if comp in x else False)].max()
sum_gen.sort_values(ascending=False).head(1)

Midnight Picture Show    1000.0
dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [62]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'  # +

In [63]:
data.sort_values(by='vote_average', ascending=False).head(round(data.shape[0]/100))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,overview_length
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008,816921825,7,15,396
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,Paramount Pictures|Legendary Pictures|Warner B...,2014-11-05,8.0,2014,456752480,11,12,222
125,tt2084970,14000000,233555708,The Imitation Game,Benedict Cumberbatch|Keira Knightley|Matthew G...,Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,Black Bear Pictures|Bristol Automotive,2014-11-14,8.0,2014,219555708,11,18,287
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,2015-06-09,8.0,2015,678708609,6,10,638
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,2015-10-16,8.0,2015,29401758,10,4,243
1183,tt0993846,100000000,392000694,The Wolf of Wall Street,Leonardo DiCaprio|Jonah Hill|Margot Robbie|Kyl...,Martin Scorsese,EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,Crime|Drama|Comedy,Paramount Pictures|Appian Way|EMJAG Production...,2013-12-25,7.9,2013,292000694,12,23,200
128,tt2267998,61000000,369330363,Gone Girl,Ben Affleck|Rosamund Pike|Carrie Coon|Neil Pat...,David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,Twentieth Century Fox Film Corporation|Regency...,2014-10-01,7.9,2014,308330363,10,9,169
1191,tt2024544,20000000,187000000,12 Years a Slave,Chiwetel Ejiofor|Michael Fassbender|Lupita Nyo...,Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,Drama|History,Plan B Entertainment|Regency Enterprises|River...,2013-10-18,7.9,2013,167000000,10,16,379
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,2014-07-30,7.9,2014,603312399,7,23,164
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,WingNut Films|New Line Cinema,2003-12-01,7.9,2003,1024888979,12,45,254


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [64]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'  # +

In [65]:
data.cast.str.split('|').apply(lambda x: list(combinations(x, 2))).explode('cast').value_counts()[:1]

(Daniel Radcliffe, Emma Watson)    8
Name: cast, dtype: int64

# Submission

In [66]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [67]:
# и убедиться что ни чего не пропустил)
len(answers)

27